In [ ]:
# Hilfscode, um die Premium-Texte zu testen (kein direkter Einfluss auf App)
brewer_tier = "brewer"
bypass_limits = False

if not bypass_limits and brewer_tier == "brewer":
    print("Brewer Tier: AI ja, Limits nein.")
elif not bypass_limits:
    print("Free Tier: Limits gelten.")
else:
    print("Unlimited.")

In [1]:
def calculate_abv(og_plato, fg_plato):
    """
    Berechnet den Alkoholgehalt (Vol%) basierend auf Stammwürze (°P) und Restextrakt (°P).
    Verwendet eine Standard-Näherungsformel.
    """
    if og_plato is None or fg_plato is None:
        return 0.0
    
    # Näherungsformel : (Stammwürze - Restextrakt) / (2.0665 - 0.010665 * Stammwürze)
    if og_plato == 0: return 0
    return (og_plato - fg_plato) / (2.0665 - 0.010665 * og_plato)

# Test
print(f"12°P -> 3°P = {calculate_abv(12, 3):.2f}%")
print(f"16°P -> 4°P = {calculate_abv(16, 4):.2f}%")

12°P -> 3°P = 4.64%
16°P -> 4°P = 6.33%


In [2]:
import math

def calculate_ebc(batch_volume_liters, malts):
    """
    Calculates EBC color using Morey's formula adapted for metric units.
    malts: list of dicts { 'amount': kg, 'color_ebc': ebc }
    """
    if not batch_volume_liters or batch_volume_liters <= 0:
        return 0
    
    mcu_total = 0
    for m in malts:
        # Convert KG to Lbs (1kg = 2.20462 lbs)
        # Convert EBC to Lovibond (Lovibond = (EBC * 0.508) roughly or (EBC / 1.97)) -> Formula usually uses specific conversions
        # Actually, let's look at the MCU imperial definition: (Weight_lbs * Color_L) / Vol_gal
        
        weight_lbs = m['amount'] * 2.20462
        # EBC to Lovibond approximation: L = EBC / 1.97
        color_lovibond = m['color_ebc'] / 1.97
        vol_gal = batch_volume_liters * 0.264172
        
        mcu = (weight_lbs * color_lovibond) / vol_gal
        mcu_total += mcu
        
    # Morey Equation for SRM: SRM = 1.4922 * (MCU ^ 0.6859)
    srm = 1.4922 * (mcu_total ** 0.6859)
    
    # Convert SRM back to EBC: EBC = SRM * 1.97
    return srm * 1.97

def calculate_ibu(batch_volume_liters, boil_gravity, hops):
    """
    Calculates IBU using Tinseth formula.
    hops: list of dicts { 'amount': g, 'alpha': %, 'time': min, 'usage': 'Boil' }
    boil_gravity: SG (e.g., 1.050)
    """
    if not batch_volume_liters or batch_volume_liters <= 0:
        return 0
        
    total_ibu = 0
    
    for h in hops:
        if h.get('usage') != 'Boil' and h.get('usage') != 'First Wort': 
            continue # Skip dry hop / mash for bitterness contribution (simplified)
            
        amount_g = h['amount']
        alpha = h['alpha']
        time = h['time']
        
        # Tinseth Components
        
        # 1. Bigness Factor (Gravity effect)
        # 1.65 * 0.000125^(Gb - 1)
        bigness = 1.65 * (0.000125 ** (boil_gravity - 1))
        
        # 2. Boil Time Factor
        # (1 - e^(-0.04 * time)) / 4.15
        boil_factor = (1 - math.exp(-0.04 * time)) / 4.15
        
        utilization = bigness * boil_factor
        
        # IBUs = (Utilization * mg_alpha_acid) / Liters
        # mg_alpha = weight_g * alpha_percent * 10 
        # (e.g. 5% alpha in 10g hop -> 0.5g alpha = 500mg. 10g * 5 * 10 is wrong? 10g * 0.05 = 0.5g = 500mg. So it's g * (alpha/100) * 1000 = g * alpha * 10. Yes.)
        mg_alpha = amount_g * alpha * 10
        
        ibu = (utilization * mg_alpha) / batch_volume_liters
        total_ibu += ibu
        
    return total_ibu

# Test Data
malts_test = [
    {'amount': 4.5, 'color_ebc': 3}, # Pilsner
    {'amount': 0.5, 'color_ebc': 20} # Munich
]
hops_test = [
    {'amount': 30, 'alpha': 6.0, 'time': 60, 'usage': 'Boil'},
    {'amount': 20, 'alpha': 3.0, 'time': 10, 'usage': 'Boil'}
]
vol = 20
og = 1.050

print(f"Calculated EBC: {calculate_ebc(vol, malts_test):.1f}")
print(f"Calculated IBU: {calculate_ibu(vol, og, hops_test):.1f}")


Calculated EBC: 8.8
Calculated IBU: 23.3
